In [1]:
import torch 
import torch.nn as nn 
import numpy as np
from collections import OrderedDict

from utils.losses import protop_loss
from utils.sample_parameters import SingleGenerator, HiddenAndKernelGenerator, MonotonicGenerator
from utils.sample_parameters import ParamGenerators
from models.ProtICU import ProtICU
from utils.train_n_test import TrainTest

In [2]:
train, val, test = torch.load('data/in-hospital-mortality/tensors.pkl')

In [3]:
# Calculating class weights
perc_mort = np.concatenate((train[1], val[1], test[1])).mean()
class_weights = torch.Tensor([perc_mort, 1-perc_mort])

In [14]:
generators_dict = {
    # HYPERMARAMETER RANGES
    'BATCH_SIZE': SingleGenerator([128, 256, 512]),
    'EPOCHS': SingleGenerator([20, 40, 50]),
    'OPTIMIZER': SingleGenerator([torch.optim.Adam]),
    'LEARNING_RATE': SingleGenerator([1e-5, 5e-5, 1e-4, 5e-4]),
    'LOSS': SingleGenerator([protop_loss(class_weights, .5 , .5), protop_loss(class_weights, 1, 1),
                             protop_loss(class_weights, .1 , .1), protop_loss(class_weights, 1 , .5),
                             protop_loss(class_weights, .5 , .1), protop_loss(class_weights, .3 , .1)]),
    'EARLY_STOPPING': SingleGenerator([True]),
    'PATIENCE': SingleGenerator(list(range(2,4))),
    'MIN_DELTA': SingleGenerator([5e-5, 1e-4, 5e-4, 1e-3, 5e-3]), 

    # NETWORK PARAMETER RANGES
    'HIDDEN_AND_KERNEL_SIZES': HiddenAndKernelGenerator(range(1,4), [64, 128, 256, 512], [3, 5, 7, 9], 
                               ascending=(True, False)),
    'MAXPOOL': SingleGenerator([2]),
    'OBO_SIZES': MonotonicGenerator(range(1,3), [64, 128, 256, 512], ascending=False),
    'PROTOTYPE_NUM': SingleGenerator([10,20]),
    'DROPOUT': SingleGenerator(np.arange(8)/10)
} # each of these params are sampled INDEPENDENTLY of one another

In [16]:
N = 30
gen = ParamGenerators(generators_dict)
param_samples = gen.sample(N)

In [7]:
stats = []
for i in np.arange(N):
    print(param_samples[i])
    run = TrainTest(ProtICU, (train, val, test), param_samples[i])
    run.train()
    stats.append(run.test())

  0%|          | 0/115 [00:00<?, ?it/s]

{'BATCH_SIZE': 128, 'EPOCHS': 40, 'OPTIMIZER': <class 'torch.optim.adam.Adam'>, 'LEARNING_RATE': 5e-05, 'LOSS': protop_loss_0.5_0.5, 'EARLY_STOPPING': True, 'PATIENCE': 2, 'MIN_DELTA': 0.0001, 'MAXPOOL': 2, 'OBO_SIZES': array([256, 256]), 'PROTOTYPE_NUM': 20, 'DROPOUT': 0.5, 'HIDDEN_SIZES': array([ 64, 128]), 'KERNEL_SIZES': array([7, 3])}


  1%|          | 1/115 [00:00<00:14,  8.00it/s]

Epoch: 0, train_loss: 0.29564544558525085, valid_loss: -0.20622141659259796


  0%|          | 0/115 [00:00<?, ?it/s]

...

val loss increased, patience count:  1
Epoch: 12, train_loss: -0.809702455997467, valid_loss: -0.7843737006187439


  1%|          | 1/115 [00:00<00:13,  8.43it/s]

val loss increased, patience count:  2
Epoch: 13, train_loss: -0.8138859868049622, valid_loss: -0.7818833589553833


100%|██████████| 16/16 [00:00<00:00, 18.43it/s]


Early stopped at Epoch:  14


In [8]:
stats

[{'epoch_stopped': 14,
  'auroc': 0.7195216900196326,
  'auprc': 0.3724263590994855,
  'acc': 0.8538315988647115,
  'f1': 0.0}]

In [9]:
for i in np.arange(N):
    for key,val in stats[i].items():
        param_samples[i][key] = val

In [ ]:
torch.save(param_samples, 'results/Proto_experiment_nopush_N'+str(N)+'.pkl')

In [57]:
model = ProtICU(test[0].shape, 2, [64, 128], [5,3], 2, [256, 128], [0,0,0,0,0,1,1,1,1,1], .2)
out, min_dis = model(test[0][:14])